In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt


def kernel_pca(indep_X, dep_Y, n_components, kernel):
    kpca = KernelPCA(n_components=n_components, kernel=kernel)
    indep_X_kpca = kpca.fit_transform(indep_X)
    return indep_X_kpca

    
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
 
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

def logistic(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'linear', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
   
def Navie(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.naive_bayes import GaussianNB
        classifier = GaussianNB()
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm         
    
    
def knn(X_train,y_train,X_test):
           
        # Fitting K-NN to the Training set
        from sklearn.neighbors import KNeighborsClassifier
        classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.tree import DecisionTreeClassifier
        classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm      


def random(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestClassifier
        classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm
    
def KernelPCA_Classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf): 
    
    kpcadataframe=pd.DataFrame(index=['KPCA'],columns=['Logistic','SVMl','SVMnl',
                                                                                        'KNN','Navie','Decision','Random'])

    for number,idex in enumerate(kpcadataframe.index):
        
        kpcadataframe['Logistic'][idex]=acclog[number]       
        kpcadataframe['SVMl'][idex]=accsvml[number]
        kpcadataframe['SVMnl'][idex]=accsvmnl[number]
        kpcadataframe['KNN'][idex]=accknn[number]
        kpcadataframe['Navie'][idex]=accnav[number]
        kpcadataframe['Decision'][idex]=accdes[number]
        kpcadataframe['Random'][idex]=accrf[number]
    return kpcadataframe

    

In [2]:
dataset=pd.read_csv("PP_Mtrls_and_their_Mchncl_Prprts.csv")

dataset.drop('Material', axis=1, inplace=True)
dataset['Use'] = dataset['Use'].astype('object')
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Su      802 non-null    int64  
 1   Sy      802 non-null    int64  
 2   E       802 non-null    int64  
 3   G       802 non-null    int64  
 4   mu      802 non-null    float64
 5   Ro      802 non-null    int64  
 6   Use     802 non-null    object 
dtypes: float64(1), int64(5), object(1)
memory usage: 44.0+ KB


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)
indep_X=dataset.drop('Use_True',axis= 1)
dep_Y=dataset['Use_True']

dataset.head()

,Su,Sy,E,G,mu,Ro,Use_True
0,421,314,207000,79000,0.3,7860,True
1,424,324,207000,79000,0.3,7860,True
2,386,284,207000,79000,0.3,7860,True
3,448,331,207000,79000,0.3,7860,True
4,441,346,207000,79000,0.3,7860,True


In [8]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']  
n_components = 5
kernel_results = {}  

for kernel in kernels:
    indep_X_kpca = kernel_pca(indep_X, dep_Y, n_components, kernel)
    
    # Check if the length of indep_X_kpca matches your original dataset
   
    if len(indep_X_kpca) != len(dep_Y):
        print(f"Length mismatch for {kernel} kernel. Skipping.")
        continue

    acclog = []
    accsvml = []
    accsvmnl = []
    accknn = []
    accnav = []
    accdes = []
    accrf = []

    X_train, X_test, y_train, y_test = split_scalar(indep_X_kpca, dep_Y)

    classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test)
    acclog.append(Accuracy)

    classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train, y_train, X_test)
    accsvml.append(Accuracy)

    classifier, Accuracy, report, X_test, y_test, cm = svm_NL(X_train, y_train, X_test)
    accsvmnl.append(Accuracy)

    classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test)
    accknn.append(Accuracy)

    classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test)
    accnav.append(Accuracy)

    classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test)
    accdes.append(Accuracy)

    classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test)
    accrf.append(Accuracy)

    result = KernelPCA_Classification(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)

    #kernel_results[kernel] = result

# Access the results for each kernel from kernel_results dictionary
#print(kernel_results)


/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/viswanathanmuthu/anaconda3/envs/AIVE_Course/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

In [5]:
#3
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
KPCA,0.950249,0.950249,0.950249,0.950249,0.950249,0.950249,0.950249


In [7]:
#4
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
KPCA,0.950249,0.950249,0.950249,0.950249,0.950249,0.950249,0.950249


In [9]:
#5
result

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
KPCA,0.950249,0.950249,0.950249,0.950249,0.950249,0.950249,0.950249
